# CamVid Dataset Preprocessing

This notebook preprocesses the [CamVid (Cambridge-driving Labeled Video Database)](http://mi.eng.cam.ac.uk/research/projects/VideoRec/CamVid/)
dataset (made avialable for download on Kaggle at 
[https://www.kaggle.com/datasets/carlolepelaars/camvid](https://www.kaggle.com/datasets/carlolepelaars/camvid)).

As a prerequisite for this notebook, please download the dataset from Kaggle and extract the `archive.zip` file.

## Module imports and Amazon SageMaker setup

Here we import modules from the [Amazon SageMaker Python SDK](https://sagemaker.readthedocs.io/en/stable/)
that we need for preprocessing the data.

We then create a SageMaker session, set the execution role,
and define the S3 bucket and prefix where data and artifacts are stored.

In [3]:
%reload_ext autoreload
%autoreload 2
%reload_ext dotenv
%dotenv

import sagemaker
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.processing import ProcessingInput, ProcessingOutput

session = sagemaker.Session()
# By default the bucket name follows the format "sagemaker-{region}-{aws-account-id}"
bucket = session.default_bucket()
# You can adapt the prefix to your liking. Just make sure to also update it in all other notebooks as well.
prefix = 'enet-tensorflow-distributed'
role = sagemaker.get_execution_role()

## Data Paths

Next we set the S3 URIs for the raw data and where the preprocessed data will be stored.
This uses the bucket and prefix that were set in the previous cell.

The `input_path` points to the raw dataset location in S3.

🚨 Please upload the raw dataset to this location in S3 before continuing with this notebook. 🚨

In [4]:
input_path = f's3://{bucket}/{prefix}/raw-data/CamVid/'
train_path = f's3://{bucket}/{prefix}/preprocessed-data/camvid/train/'
train_labels_path = f's3://{bucket}/{prefix}/preprocessed-data/camvid/train_labels/'
val_path = f's3://{bucket}/{prefix}/preprocessed-data/camvid/val/'
val_labels_path = f's3://{bucket}/{prefix}/preprocessed-data/camvid/val_labels/'
test_path = f's3://{bucket}/{prefix}/preprocessed-data/camvid/test/'
test_labels_path = f's3://{bucket}/{prefix}/preprocessed-data/camvid/test_labels/'
report_path = f's3://{bucket}/{prefix}/preprocessed-data/camvid/report/'
preprocessing_report_path = f'{report_path}preprocessing_report.json'
class_dict_path = f'{report_path}class_dict.json'

## Define Preprocessing Job

The data preprocessing is done via the [`SKLearnProcessor`](https://sagemaker.readthedocs.io/en/stable/frameworks/sklearn/sagemaker.sklearn.html#scikit-learn-processor) which is defined in the next cell.

In [ ]:
sklearn_processor = SKLearnProcessor(
    framework_version='0.23-1',
    role=role,
    instance_type='ml.m5.large',
    instance_count=1, 
    base_job_name='camvid-preprocessing'
)

## Run Processing Job

The processing job is now run. The processing code is located at [`../datasets/camvid/preprocess.py`](../datasets/camvid/preprocess.py). The processing outputs are stored in S3 at the previously defined locations.

In [ ]:
sklearn_processor.run(
    code='../datasets/camvid/preprocess.py',
    inputs=[
        ProcessingInput(
            input_name='raw_dataset',
            source=input_path, 
            destination='/opt/ml/processing/input',
            s3_input_mode='File',
            s3_data_distribution_type='ShardedByS3Key'
        )
    ],
    outputs=[
        ProcessingOutput(
            output_name='train',
            source='/opt/ml/processing/output/train',
            destination=train_path,
        ),
        ProcessingOutput(
            output_name='train_labels',
            source='/opt/ml/processing/output/train_labels',
            destination=train_labels_path,
        ),
        ProcessingOutput(
            output_name='val',
            source='/opt/ml/processing/output/val',
            destination=val_path,
        ),
        ProcessingOutput(
            output_name='val_labels',
            source='/opt/ml/processing/output/val_labels',
            destination=val_labels_path,
        ),
        ProcessingOutput(
            output_name='test',
            source='/opt/ml/processing/output/test',
            destination=test_path,
        ),
        ProcessingOutput(
            output_name='test_labels',
            source='/opt/ml/processing/output/test_labels',
            destination=test_labels_path,
        ),
        ProcessingOutput(
            output_name='report',
            source='/opt/ml/processing/report',
            destination=report_path,
        ),
    ]
)